In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Other libraries using Matplotlib

- Matplotlib provides building blocks and configuration capabilities to adapt a plot completely to your needs.
- That's usually a couple of lines of code.
- Providing ready made domain-specific plots is beyond the scope.

There are many libraries building on top of Matplotlib https://matplotlib.org/thirdpartypackages/index.html

## Write your own wrapper function

Generally, you have two types:

- Functions that create a complete figure
- Functions that plot data into an existing axes


### Creating a whole figure

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict({
    'Question 1': [10, 15, 17, 32, 26],
    'Question 2': [26, 22, 29, 10, 13],
    'Question 3': [35, 37, 7, 2, 19],
    'Question 4': [32, 11, 9, 15, 33],
    'Question 5': [21, 29, 5, 5, 40],
    'Question 6': [8, 19, 5, 30, 38]
    },
    orient='index',
    columns=[
        'Strongly disagree', 'Disagree', 'Neither agree nor disagree', 'Agree', 'Strongly agree']
)
df

Recommendation:

- **Inputs:** The data  
  Optionally add config parameters to make the plot more customizable.
- **Output:** The created `Figure` and `Axes` (or `Axes`es), often the same as the `subplots()` return value.  
  This gives users access to the fundamental objects and allows further customization.
  

Example: https://matplotlib.org/stable/gallery/lines_bars_and_markers/horizontal_barchart_distribution.html

In [ ]:
def survey(df):
    """
    Parameters
    ----------
    df : DataFrame
        A DataFrame with one row per questiong and the agreement categories
        in the columns.
    """
    data = df.to_numpy()
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 5))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(df.columns, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(df.index, widths, left=starts, height=0.5,
                        label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        ax.bar_label(rects, label_type='center', color=text_color)
    ax.legend(ncol=len(category_names), bbox_to_anchor=(0, 1),
              loc='lower left', fontsize='small')

    return fig, ax

In [ ]:
fig, ax = survey(df)

fig.suptitle('Test')

### Helper function to plot data

- **Inputs:**
  - The `Axes` to plot into.
  - The data.
  - Addtional configuration parameters. - It's often reasonable to pass through arbitrary parameters using `**kwargs`.
- **Output:** Usually not needed, but it may be a good idea to return the created `Artist`s - like the builtin plot functions do.


In [ ]:
def plot_errorband(ax, x, y, err, **kwargs):
    """
    Plot a line (x, y) and the corresponding error band y +/- err into the given Axes *ax*.
    """
    poly = ax.fill_between(x, y-err, y+err, alpha=0.5, **kwargs)
    line = ax.plot(x, y, **kwargs)
    return line, poly

In [ ]:
# data
np.random.seed(1)
x = np.linspace(0, 8, 16)
y = 3 + 4*x/8 + np.random.uniform(0.0, 0.5, len(x))
err = 5 + np.random.normal(size=len(x))

# plot
fig, ax = plt.subplots()
plot_errorband(ax, x, y, err)
plot_errorband(ax, x, y+10, err, color='green')  # color='green'

#### Alternative: more `pyplot`-like interface

Make the `Axes` *ax* an optional parameter and use `plt.gca()` if not provided.

In [ ]:
def plot_errorband(x, y, err, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    
    poly = ax.fill_between(x, y-err, y+err, alpha=0.5, **kwargs)
    line = ax.plot(x, y, **kwargs)
    return line, poly

In [ ]:
plot_errorband(x, y, err)
plot_errorband(x, y+10, err)

# `pandas` and matplotlib

`pandas.plot` provides a high-level plotting interface to draw content of DataFrames using Matplotllib.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns

In [ ]:
df = pd.DataFrame({'A': [1, 2, 4, 8], 'B': [4, 6, 3, 5]})

In [ ]:
df

Pure Matplotlib:

In [ ]:
plt.plot(df.index, df.A)

Using `DataFrame.plot`.

In [ ]:
df.plot.line(y='A')

*Note*: `DataFrame.plot` has originally been a single function. The kind of plot (line, bar, ...) could be chosen by the `kind` parameter. This is a bit too generic.

Nowadays, `DataFrame.plot` can also be regarded as a namespace with sub-functions `DataFrame.plot.line()`, `DataFrame.plot.bar()`, ...

### Which Axes do my data go to?

Handling of Axes:

- `matplotlib.pyplot`: Plot into current axes (create one if necessary)
- `matplotlib` OOP: Explicitly create axes
- `pandas.plot`: Creates a new axes and plots into that. Alternatively, pass an existing axes via the `ax` keyword argument.

Return value:
- `matplotlib` plotting functions return the created Artist.
- `pandas.plot` plotting functions return the axes

In [ ]:
fig, axs = plt.subplots(1, 2)
df.plot.line(y='A')
#df.plot.line(y='A', ax=axs[1])

### Philosophies

- Matplotlib: Every aspect of the plot is configured by a separate function.
- `pandas.plot`: Convenience function to quickly draw data with some common settings.

In [ ]:
df.plot.line(ylim=(0, None), title='My data')

In [ ]:
plt.plot(df.index, df.A, label='A')
plt.plot(df.index, df.B, label='B')
plt.ylim(0, None)
plt.xlim(0, max(df.index))
plt.legend()
plt.title('My data')

You combine both worlds:
- Use conveninece functions to draw the main part of the figure.
- Fine-tune further using Matplotlib functions.

In [ ]:
ax = df.plot.line(ylim=(0, None), marker='o', title='My data')
ax.annotate('Look here', (1, 6), xytext=(1.3, 7.2), arrowprops={'arrowstyle': '->'})

In [ ]:
df.index = [f'Row {i}' for i in df.index]

In [ ]:
df.plot.bar(rot=45)